In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

import warnings
warnings.filterwarnings("ignore")

import re

from pymongo import MongoClient
import pandas as pd

In [12]:
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    
opciones.add_argument('--start-maximized')        
opciones.add_argument('user-data-dir=selenium')    
opciones.add_argument('--incognito')         

In [13]:
driver = webdriver.Chrome("./chromedriver.exe")
url= "https://fxtop.com/es/lista-de-paises.php"
driver.get(url)
driver.implicitly_wait(5)
driver.find_element_by_css_selector("body > div > div.fc-dialog-container > div.fc-dialog.fc-choice-dialog > div.fc-footer-buttons-container > div.fc-footer-buttons > button.fc-button.fc-cta-consent.fc-primary-button").click()

dicc={"Countries":[],"Divisas":[],"Abrev":[]}

for i in range(2,245):
    dicc["Countries"].append(driver.find_element_by_css_selector(f"body > table > tbody > tr > td:nth-child(2) > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(5) > td > table > tbody > tr:nth-child({i}) > td:nth-child(1)").text)
    dicc["Divisas"].append(driver.find_element_by_css_selector(f"body > table > tbody > tr > td:nth-child(2) > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(5) > td > table > tbody > tr:nth-child({i}) > td:nth-child(3)").text)
    dicc["Abrev"].append(driver.find_element_by_css_selector(f"body > table > tbody > tr > td:nth-child(2) > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(5) > td > table > tbody > tr:nth-child({i}) > td:nth-child(4)").text)

driver.quit()

In [14]:
div=pd.DataFrame(dicc)

In [15]:
div.head()

,Countries,Divisas,Abrev
0,Martinica,EUR euro,EUR / EUR
1,Afganistán,AFN afgani afgano,EUR / AFN
2,África del Sur,ZAR rand sudafricano,EUR / ZAR
3,Albania,ALL lek albanés,EUR / ALL
4,Alemania,EUR euro,EUR / EUR


In [16]:
abr=[]
for fila in div.Abrev:
    abr.append(fila.split(" / ")[1])
    

In [17]:
div['Abrev']=abr

In [18]:
div.head()

,Countries,Divisas,Abrev
0,Martinica,EUR euro,EUR
1,Afganistán,AFN afgani afgano,AFN
2,África del Sur,ZAR rand sudafricano,ZAR
3,Albania,ALL lek albanés,ALL
4,Alemania,EUR euro,EUR


In [19]:
divisas=list(div.Divisas)

In [29]:
divisas_lista=[]
for d in divisas:
    elemento=re.findall("[a-z].*",d)
    divisas_lista.append(elemento[0])

In [30]:
div["Divisas"]=divisas_lista

In [31]:
div.head()

,Countries,Divisas,Abrev
0,Martinica,euro,EUR
1,Afganistán,afgani afgano,AFN
2,África del Sur,rand sudafricano,ZAR
3,Albania,lek albanés,ALL
4,Alemania,euro,EUR


In [67]:
div.Countries = div.Countries.str.replace("España", "Espana")

In [68]:
div.to_csv("../Data/divisasdf.csv")

In [40]:
def get_country(city):
    """
    Esta función devuelve el país de una ciudad dada.
    Args: Ciudad (string).
    Return: País (string).
    """
    client = MongoClient("localhost:27017")
    db = client.get_database("Tu_viaje_ideal_en_un_click")
    c = db.get_collection("coordenadas")
    cond={"City": city}
    #pr={"IATA": 1,"Country":0, "_id":0, "City":1, "Location":0}
    div=list(c.find(cond))
    paises=div[0]['Country']
    
    return paises

In [57]:
get_country("Madrid")

'Alemania'

In [66]:
list(div[div["Countries"] == "Espana"]["Abrev"])[0]

'EUR'

In [3]:
def load_data():
    return pd.read_csv("../Data/divisasdf.csv")

In [4]:
load_data()

,Unnamed: 0,Countries,Divisas,Abrev
0,0,Martinica,euro,EUR
1,1,Afganistán,afgani afgano,AFN
2,2,África del Sur,rand sudafricano,ZAR
3,3,Albania,lek albanés,ALL
4,4,Alemania,euro,EUR
...,...,...,...,...
238,238,Yemen,rial yemeni,YER
239,239,Yibuti,franco yibutiano,DJF
240,240,Yugoslavia,dinar yugoslavo,YUN
241,241,Zambia,kwacha zambiano,ZMW
